# Importing Relevant Packages

In [1]:
import numpy as np
import pandas as pd 
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tensorflow_datasets as tfds

# Getting the Mnist Data

In [2]:
mnist_dataset, mnist_info = tfds.load(name = 'mnist', with_info = True, as_supervised = True)

# Preprocessing

In [113]:
mnist_train, mnist_test = mnist_dataset['train'],mnist_dataset['test']

num_val_samples = .1 * mnist_info.splits['train'].num_examples
num_val_samples = tf.cast(num_val_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_val_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_val_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

Buffer_size = 20 #Value chosen from "Further Improvments" section

shuffled_train_and_val_data = scaled_train_and_val_data.shuffle(Buffer_size)

val_data = shuffled_train_and_val_data.take(num_val_samples)
train_data = shuffled_train_and_val_data.skip(num_val_samples)

Batch_size = 50 #Value chosen from "Further Improvments" section

train_data = train_data.batch(Batch_size)
val_data = val_data.batch(num_val_samples)
test_data = test_data.batch(num_test_samples)

val_inputs, val_targets = next(iter(val_data))

# Model

In [114]:
input_size = 784
output_size = 10
hidden_layer_size = 200 #Value chosen from "Further Improvments" section

model = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape = (28,28,1)),
            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
            tf.keras.layers.Dense(output_size, activation = 'softmax')
            ])

# Optimizer and Loss Function

In [115]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

# Training

In [116]:
num_epochs = 5
stop = tf.keras.callbacks.EarlyStopping(patience= 2)
model.fit(train_data, epochs= num_epochs, callbacks = stop,
          validation_data = (val_inputs,val_targets),verbose = 1)

Epoch 1/5
1080/1080 [==============================] - 10s 8ms/step - loss: 0.4532 - accuracy: 0.8708 - val_loss: 0.1479 - val_accuracy: 0.9558
Epoch 2/5
1080/1080 [==============================] - 8s 7ms/step - loss: 0.1224 - accuracy: 0.9649 - val_loss: 0.1125 - val_accuracy: 0.9643
Epoch 3/5
1080/1080 [==============================] - 8s 7ms/step - loss: 0.0792 - accuracy: 0.9770 - val_loss: 0.1011 - val_accuracy: 0.9680
Epoch 4/5
1080/1080 [==============================] - 7s 6ms/step - loss: 0.0566 - accuracy: 0.9840 - val_loss: 0.0960 - val_accuracy: 0.9700
Epoch 5/5
1080/1080 [==============================] - 8s 7ms/step - loss: 0.0415 - accuracy: 0.9891 - val_loss: 0.0977 - val_accuracy: 0.9712


# Further Improvements
### We will use nested for loops of hyperperameters to see what works best before evaluating our test set

In [104]:
col_names = ['Buffer_size', 'Batch_size','hidden_layer_size', 'val_accuracy']
results_df = pd.DataFrame(columns= col_names)
for hidden_layer_size in list(range(150, 300, 50)):
    for Batch_size in list(range(25, 150, 25)): 
        shuffled_train_and_val_data = scaled_train_and_val_data.shuffle(Buffer_size)
        
        val_data = shuffled_train_and_val_data.take(num_val_samples)
        train_data = shuffled_train_and_val_data.skip(num_val_samples)

        train_data = train_data.batch(Batch_size)
        val_data = val_data.batch(num_val_samples)
        test_data = test_data.batch(num_test_samples)

        val_inputs, val_targets = next(iter(val_data))

        model = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape = (28,28,1)),
            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
            tf.keras.layers.Dense(output_size, activation = 'softmax')
            ])
        model.compile(optimizer = 'adam',
                    loss = 'sparse_categorical_crossentropy',
                    metrics = ['accuracy'])
        num_epochs = 3
        stop = tf.keras.callbacks.EarlyStopping(patience= 2)
        results = model.fit(train_data, epochs= num_epochs, callbacks = stop,
        validation_data = (val_inputs,val_targets),verbose = 0)

        val_accuracy = results.history['val_accuracy'][len(results.history['val_accuracy']) -1]
        to_append = [Buffer_size, Batch_size, hidden_layer_size ,val_accuracy]
        df_length =  len(results_df)
        results_df.loc[df_length] = to_append
            

In [105]:
#As shown, since there is s
results_df.sort_values('val_accuracy', ascending= False)

,Buffer_size,Batch_size,hidden_layer_size,val_accuracy
6,20.0,50.0,200.0,0.971667
9,20.0,125.0,200.0,0.970833
1,20.0,50.0,150.0,0.970500
14,20.0,125.0,250.0,0.969833
10,20.0,25.0,250.0,0.969667
5,20.0,25.0,200.0,0.969333
8,20.0,100.0,200.0,0.969333
11,20.0,50.0,250.0,0.969167
7,20.0,75.0,200.0,0.969000
13,20.0,100.0,250.0,0.968333


# Testing our Model

In [89]:
test_loss, test_accuracy = model.evaluate(test_data)

2/2 [==============================] - 1s 235ms/step - loss: 0.0863 - accuracy: 0.9748


# Conclusion:
### Our model has an accuracy of 97%
### Thus, given 10 new customers, we can accuratly predict whether 9 of those customers will continue to use the app or not

In [117]:
model.evaluate(test_data)

2/2 [==============================] - 1s 310ms/step - loss: 0.0828 - accuracy: 0.9743


[0.08278011530637741, 0.9743000268936157]